In [2]:
import pandas as pd
import geopandas as gp 
import json
from glob import glob
import numpy as np
import numpy as np
from tqdm import tqdm

In [3]:
ward_pop = pd.read_csv('../mappings/ward_pop.csv')

/home/sagar/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (23,24,25,26,27,28,29,30,31,32,33,34,35) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
%store -r cityMap

In [6]:
population = {}
for index, row in ward_pop.iterrows():
    population[row['Ward Code 1']] = float(row['All Ages'].replace(',',''))

In [14]:
df_LSOA_CITY = pd.read_csv('mappings/lower_layer_super_output_area_2011_to_major_towns_and_cities_december_2015_lookup_in_england_and_wales.csv')

In [15]:
df_LSOA_WARD = pd.read_csv('mappings/lower_layer_super_output_area_(2011)_to_ward_(2015)_lookup_in_england_and_wales.csv')

In [16]:
df_LSOA_WARD.head()

,LSOA11CD,LSOA11NM,WD15CD,WD15NM,WD15NMW,LAD15CD,LAD15NM,FID
0,E01000104,Barking and Dagenham 014B,E05000041,Village,,E09000002,Barking and Dagenham,1
1,E01000105,Barking and Dagenham 014C,E05000041,Village,,E09000002,Barking and Dagenham,2
2,E01000106,Barking and Dagenham 014D,E05000041,Village,,E09000002,Barking and Dagenham,3
3,E01000107,Barking and Dagenham 010C,E05000041,Village,,E09000002,Barking and Dagenham,4
4,E01000108,Barking and Dagenham 010D,E05000041,Village,,E09000002,Barking and Dagenham,5


In [18]:
df_LSOA_CITY.head()

,LSOA11CD,LSOA11NM,TCITY15CD,TCITY15NM,FID
0,E01002351,Havering 016C,J01000055,London,2001
1,E01002352,Havering 016D,J01000055,London,2002
2,E01002100,Haringey 008B,J01000055,London,2003
3,E01002301,Havering 003A,J01000055,London,2004
4,E01002353,Havering 013B,J01000055,London,2005


In [19]:
mega_df = pd.merge(df_LSOA_WARD,df_LSOA_CITY,left_on='LSOA11CD' , right_on='LSOA11CD')

In [20]:
mega_df.head()

,LSOA11CD,LSOA11NM_x,WD15CD,WD15NM,WD15NMW,LAD15CD,LAD15NM,FID_x,LSOA11NM_y,TCITY15CD,TCITY15NM,FID_y
0,E01000104,Barking and Dagenham 014B,E05000041,Village,,E09000002,Barking and Dagenham,1,Barking and Dagenham 014B,J01000055,London,151
1,E01000105,Barking and Dagenham 014C,E05000041,Village,,E09000002,Barking and Dagenham,2,Barking and Dagenham 014C,J01000055,London,152
2,E01000106,Barking and Dagenham 014D,E05000041,Village,,E09000002,Barking and Dagenham,3,Barking and Dagenham 014D,J01000055,London,153
3,E01000107,Barking and Dagenham 010C,E05000041,Village,,E09000002,Barking and Dagenham,4,Barking and Dagenham 010C,J01000055,London,154
4,E01000108,Barking and Dagenham 010D,E05000041,Village,,E09000002,Barking and Dagenham,5,Barking and Dagenham 010D,J01000055,London,155


In [21]:
sub_data = mega_df[['LSOA11CD','WD15CD','WD15NM','LAD15CD','LAD15NM','TCITY15NM']].copy()

In [24]:
sub_data.rename(columns={'LAD15NM':'LocalAuthorityName' , 'TCITY15NM':'CityName'},inplace=True)

In [27]:
sub_data.CityName = sub_data.CityName.fillna('')

In [28]:
sub_data

,LSOA11CD,WD15CD,WD15NM,LAD15CD,LocalAuthorityName,CityName
0,E01000104,E05000041,Village,E09000002,Barking and Dagenham,London
1,E01000105,E05000041,Village,E09000002,Barking and Dagenham,London
2,E01000106,E05000041,Village,E09000002,Barking and Dagenham,London
3,E01000107,E05000041,Village,E09000002,Barking and Dagenham,London
4,E01000108,E05000041,Village,E09000002,Barking and Dagenham,London
...,...,...,...,...,...,...
34748,W01001911,W05000635,Castleland,W06000014,Vale of Glamorgan,
34749,W01001860,W05000877,Riverside,W06000015,Cardiff,Cardiff
34750,W01001753,W05000861,Fairwater,W06000015,Cardiff,Cardiff
34751,W01001754,W05000861,Fairwater,W06000015,Cardiff,Cardiff


In [29]:
sub_data.to_csv('data_prep/Wards.csv')

In [30]:
LSOA_dist = json.load(open('../mappings/GP_LSOA_PATIENTSDIST.json','rb'))

In [31]:
LSOA_dist_2021 = json.load(open('mappings/GP_LSOA_PATIENTSDIST_2021.json','rb'))

In [37]:
LSOA_dist['Y02811'].values()

dict_values([0.0004411116012351125, 0.00011027790030877812, 0.036060873400970445, 0.03164975738861932, 0.03242170269078077, 0.018967798853109837, 0.019850022055580063, 0.012240846934274371, 0.03484781649757389, 0.014005293339214821, 0.007829730921923246, 0.00011027790030877812, 0.005513895015438906, 0.00022055580061755624, 0.0006616674018526688, 0.0003308337009263344, 0.000992501102779003, 0.0004411116012351125, 0.012240846934274371, 0.018967798853109837, 0.04378032642258491, 0.013784737538597266, 0.020401411557123952, 0.020621967357741508, 0.013012792236435818, 0.013123070136744597, 0.023930304367004853, 0.01819585355094839, 0.00011027790030877812, 0.00022055580061755624, 0.02646669607410675, 0.021393912659902956, 0.05491839435377151, 0.02095280105866784, 0.0006616674018526688, 0.0004411116012351125, 0.00011027790030877812, 0.007719453021614468, 0.00011027790030877812, 0.00011027790030877812, 0.0004411116012351125, 0.00011027790030877812, 0.0014336127040141156, 0.000882223202470225, 0

In [56]:
LSOA_dist_new = pd.read_csv('mappings/gp-reg-pat-prac-lsoa-all.csv')

In [57]:
LSOA_dist_new.head()

,PUBLICATION,EXTRACT_DATE,PRACTICE_CODE,PRACTICE_NAME,LSOA_CODE,SEX,Number of Patients
0,GP_PRAC_PAT_LIST,01APR2021,A81001,THE DENSHAM SURGERY,E01011959,ALL,7
1,GP_PRAC_PAT_LIST,01APR2021,A81001,THE DENSHAM SURGERY,E01012012,ALL,2
2,GP_PRAC_PAT_LIST,01APR2021,A81001,THE DENSHAM SURGERY,E01012013,ALL,1
3,GP_PRAC_PAT_LIST,01APR2021,A81001,THE DENSHAM SURGERY,E01012022,ALL,2
4,GP_PRAC_PAT_LIST,01APR2021,A81001,THE DENSHAM SURGERY,E01012023,ALL,2


In [79]:
LSOA_dist_2021 = {}
for name , group in tqdm(LSOA_dist_new.groupby('PRACTICE_CODE')):
    LSOA_dist_2021[name] = {}
    total = sum(group['Number of Patients'])
    LSOA_dist_2021[name]['Patient_registry_LSOA'] = {}
    LSOA_dist_2021[name]['Total_patients'] = total
    LSOA_dist_2021[name]['GP_Name'] = group['PRACTICE_NAME'].iloc[0]
    LSOA_dist_2021[name]['GP_Postcode'] = ''
    LSOA_dist_2021[name]['GP_Address'] = ''
    for index , row in group.iterrows():
        LSOA_dist_2021[name]['Patient_registry_LSOA'][row['LSOA_CODE']] = float(row['Number of Patients'])


 29%|██▉       | 1916/6623 [00:40<03:02, 25.86it/s]


 48%|████▊     | 3151/6623 [01:24<02:14, 25.84it/s]


 74%|███████▍  | 4930/6623 [02:03<00:35, 47.25it/s]


 97%|█████████▋| 6393/6623 [02:42<00:08, 26.09it/s]


100%|██████████| 6623/6623 [02:49<00:00, 39.08it/s]


In [82]:
sum(LSOA_dist_new['Number of Patients'])

60744002

In [64]:
LSOA_dist_new_codes = LSOA_dist_2021.keys()
LSOA_dist_codes = LSOA_dist.keys()
print(len(set(LSOA_dist_new_codes).intersection(LSOA_dist_codes))) , print(len(LSOA_dist_new_codes)) , print(len(LSOA_dist_codes))

6584
6623
7712


(None, None, None)

In [65]:
presc1 = pd.read_csv('../../BL_Work/openPrescribe/serialized/201108.gz',compression='gzip')

In [66]:
presc2 = pd.read_csv('../../BL_Work/openPrescribe/serialized/202101.gz',compression='gzip')

/home/sagar/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [67]:
presc2.head(n=2)

,Unnamed: 0,YEAR_MONTH,REGIONAL_OFFICE_NAME,REGIONAL_OFFICE_CODE,STP_NAME,STP_CODE,PCO_NAME,PCO_CODE,PRACTICE_NAME,PRACTICE_CODE,...,12,13,14,15,16,17,18,19,20,21
0,0,202101,NORTH WEST,Y62,CHESHIRE & MERSEYSIDE STP,QYG,KNOWSLEY CCG,01J00,ROSEHEATH SURGERY,N83031,...,NaN,NaN,NaN,27.5,0901011W0,6.000000,1.0,6.000000,AA,0.305556
1,1,202101,NORTH WEST,Y62,CHESHIRE & MERSEYSIDE STP,QYG,KNOWSLEY CCG,01J00,ROSEHEATH SURGERY,N83031,...,NaN,NaN,NaN,27.5,0901011W0,33.333333,1.0,33.333333,AA,0.055000


In [68]:
presc1.head(n=2)

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,11,12,13,14,15,16,17,18,19,20
0,0,Q30,5D7,A86003,0101010E0AAAEAE,Hydrotalcite_Oral Susp 500mg/5ml S/F,1.0,2.79,2.60,500.0,...,NaN,NaN,NaN,NaN,500.0,0101010E0,4.545455,1.0,4.545455,AA
1,1,Q30,5D7,A86003,0101010G0AAABAB,Co-Magaldrox_Susp 195mg/220mg/5ml S/F,24.0,58.45,54.63,13300.0,...,NaN,NaN,NaN,NaN,220.0,0101010G0,2660.000000,1.0,2660.000000,AA


In [69]:
filter_new_presc_new_codes = presc2[presc2['PRACTICE_CODE'].isin(LSOA_dist_new_codes)]
filter_new_presc_old_codes = presc2[presc2['PRACTICE_CODE'].isin(LSOA_dist_codes)]
filter_old_presc_new_codes = presc1[presc1['2'].isin(LSOA_dist_new_codes)]
filter_old_presc_old_codes = presc1[presc1['2'].isin(LSOA_dist_codes)]

In [70]:
filter_old_presc_old_codes['2'].nunique() , filter_old_presc_new_codes['2'].nunique() 

(6829, 6534)

In [71]:
filter_new_presc_old_codes['PRACTICE_CODE'].nunique() , filter_new_presc_new_codes['PRACTICE_CODE'].nunique() 

(5067, 5088)

In [72]:
presc2.columns

Index(['Unnamed: 0', 'YEAR_MONTH', 'REGIONAL_OFFICE_NAME',
       'REGIONAL_OFFICE_CODE', 'STP_NAME', 'STP_CODE', 'PCO_NAME', 'PCO_CODE',
       'PRACTICE_NAME', 'PRACTICE_CODE', 'ADDRESS_1', 'ADDRESS_2', 'ADDRESS_3',
       'ADDRESS_4', 'POSTCODE', 'BNF_CHEMICAL_SUBSTANCE',
       'CHEMICAL_SUBSTANCE_BNF_DESCR', 'BNF_CODE', 'BNF_DESCRIPTION',
       'BNF_CHAPTER_PLUS_CODE', 'QUANTITY', 'ITEMS', 'TOTAL_QUANTITY',
       'ADQUSAGE', 'NIC', 'ACTUAL_COST', 'UNIDENTIFIED', '11', '12', '13',
       '14', '15', '16', '17', '18', '19', '20', '21'],
      dtype='object')

In [73]:
presc2[['PRACTICE_CODE' , 'ADDRESS_1' , 'ADDRESS_2' , 'ADDRESS_3', 'ADDRESS_4', 'POSTCODE']]

,PRACTICE_CODE,ADDRESS_1,ADDRESS_2,ADDRESS_3,ADDRESS_4,POSTCODE
0,N83031,ROSEHEATH SURGERY,ROSEHEATH DRIVE,HALEWOOD,MERSEYSIDE,L26 9UH
1,N83031,ROSEHEATH SURGERY,ROSEHEATH DRIVE,HALEWOOD,MERSEYSIDE,L26 9UH
2,N83031,ROSEHEATH SURGERY,ROSEHEATH DRIVE,HALEWOOD,MERSEYSIDE,L26 9UH
3,N83031,ROSEHEATH SURGERY,ROSEHEATH DRIVE,HALEWOOD,MERSEYSIDE,L26 9UH
4,N83031,ROSEHEATH SURGERY,ROSEHEATH DRIVE,HALEWOOD,MERSEYSIDE,L26 9UH
...,...,...,...,...,...,...
13474266,N83031,ROSEHEATH SURGERY,ROSEHEATH DRIVE,HALEWOOD,MERSEYSIDE,L26 9UH
13474267,N83031,ROSEHEATH SURGERY,ROSEHEATH DRIVE,HALEWOOD,MERSEYSIDE,L26 9UH
13474268,N83031,ROSEHEATH SURGERY,ROSEHEATH DRIVE,HALEWOOD,MERSEYSIDE,L26 9UH
13474269,N83031,ROSEHEATH SURGERY,ROSEHEATH DRIVE,HALEWOOD,MERSEYSIDE,L26 9UH


In [80]:
GP_address = {}
for name , group in tqdm(presc2.groupby('PRACTICE_CODE')):
    GP_address[name] = {}
    GP_address[name]['GP_Postcode'] = group['POSTCODE'].iloc[0]
    GP_address[name]['GP_Address'] = ' '.join([group['ADDRESS_1'].iloc[0] ,group['ADDRESS_2'].iloc[0] ,group['ADDRESS_3'].iloc[0] ,group['ADDRESS_4'].iloc[0] ])


100%|██████████| 5109/5109 [00:14<00:00, 343.17it/s] 


In [84]:
for k in tqdm(GP_address):
    if k in LSOA_dist_2021:
        LSOA_dist_2021[k]['GP_Address'] = GP_address[k]['GP_Address']
        LSOA_dist_2021[k]['GP_Postcode'] = GP_address[k]['GP_Postcode']


100%|██████████| 5109/5109 [00:00<00:00, 268204.05it/s]


In [85]:
LSOA_dist_2021['N83031']

{'GP_Address': 'ROSEHEATH SURGERY ROSEHEATH DRIVE HALEWOOD MERSEYSIDE',
 'GP_Name': 'ROSEHEATH SURGERY',
 'GP_Postcode': 'L26 9UH',
 'Patient_registry_LSOA': {'E01006413': 1.0,
  'E01006419': 90.0,
  'E01006420': 159.0,
  'E01006421': 59.0,
  'E01006422': 68.0,
  'E01006423': 118.0,
  'E01006424': 2.0,
  'E01006425': 34.0,
  'E01006426': 240.0,
  'E01006427': 172.0,
  'E01006428': 267.0,
  'E01006429': 207.0,
  'E01006430': 65.0,
  'E01006431': 188.0,
  'E01006432': 478.0,
  'E01006433': 57.0,
  'E01006440': 2.0,
  'E01006443': 2.0,
  'E01006445': 2.0,
  'E01006463': 1.0,
  'E01006467': 4.0,
  'E01006475': 1.0,
  'E01006477': 4.0,
  'E01006478': 4.0,
  'E01006483': 4.0,
  'E01006497': 1.0,
  'E01006501': 1.0,
  'E01006505': 4.0,
  'E01006513': 1.0,
  'E01006521': 1.0,
  'E01006524': 2.0,
  'E01006530': 5.0,
  'E01006531': 15.0,
  'E01006533': 1.0,
  'E01006536': 5.0,
  'E01006537': 25.0,
  'E01006538': 20.0,
  'E01006540': 1.0,
  'E01006551': 2.0,
  'E01006569': 4.0,
  'E01006575': 1.0

In [86]:
len(LSOA_dist_2021.keys())

6623

In [89]:
json.dump(LSOA_dist_2021,open('data_prep/GPs.json','w'))